### Albumentations
- trochvision의 transform 보다 다양한 함수와 빠른 속도를 갖고 있습니다.
- 기본적으로 opencv와 함께 쓰이는 것 같습니다.

In [ ]:
import cv2
import albumentations as A
from albumentations.pytorch import transforms

- 아래 코드는, albumentation의 대표적인 사용방법 입니다.
- `p` 값은 해당 함수를 적용할 확률 입니다.
- `OneOf` 함수는 해당 함수 안에 들어가 있는 여러 transform 함수들 중 하나를 선택하게 합니다. 물론 `OneOf`도 마찬가지로 `p`를 설정해서 확률을 설정할 수 있습니다.

In [ ]:
# albumentation 
album_transform = A.Compose([
    A.Resize(350, 350),
    A.HorizontalFlip(p=0.7),
    A.HueSaturationValue(p=0.7),
    A.RandomBrightnessContrast(p=0.5),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
    A.OneOf([
        A.MotionBlur(p=0.7),
        A.OpticalDistortion(p=0.7),
        A.GaussNoise(p=0.7),
    ], p=1),
    A.Normalize(mean=0.5, std=0.5, max_pixel_value=1.0),
    transforms.ToTensorV2()
])

albumentation을 쓸 때는 opencv를 쓰게 되면서, DataSet 모듈의 __getitem__이 살짝 바뀝니다.

In [ ]:
 # albumentation 사용했을 경우
def __getitem__(self, idx):
    label = self.image_label.iloc[idx]
    # Read an image with OpenCV
    image = cv2.imread(self.image_data.iloc[idx])
    
    # By default OpenCV uses BGR color space for color images,
    # so we need to convert the image to RGB color space.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


    # albumentation을 적용하면서 이미지가 바뀌는 부분
    if self.transform:
        augmented = self.transform(image=image) 
        image = augmented['image']

    return image, torch.tensor(label)